In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.feature_extraction.text import CountVectorizer


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Michi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Michi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Michi\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Michi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
data = pd.read_csv("data/Recommendations.csv")
data

,Recommendation
0,Accept a lower level of service
1,Accept risks of untested scenarios and change ...
2,"Acknowledge mistakes, compensate"
3,"Acknowledge mistakes, compensate and give insi..."
4,Actively communicate status and expected resol...
...,...
256,Use the CRT feedback loop to verify effectiven...
257,Vendor must perform more tests
258,VNR / Dual SIM
259,VNR / Dual SIM


In [48]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Recommendation  261 non-null    object
dtypes: object(1)
memory usage: 2.2+ KB


In [3]:
def text_process(text):
    # Takes in a string of text, then performs the following:
    # 1. Remove all punctuation
    # 2. Remove all stopwords
    # 3. Return the cleaned text as a list of words
    # 4. Remove words
    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc =  [word.lower() for word in nopunc.split() if word not in stopwords.words('english')]
    return [stemmer.lemmatize(word) for word in nopunc]

In [4]:
def tokenize(data):
    tokenized = []
    for rec in data:
        tokenized.append(text_process(rec))
    return tokenized

In [5]:
tokens = tokenize(data["Recommendation"])
text_process(data["Recommendation"])

['accept',
 'lower',
 'level',
 'serviceaccept',
 'risk',
 'untested',
 'scenario',
 'change',
 'service',
 'level',
 'accordinglyacknowledge',
 'mistakes,',
 'compensateacknowledge',
 'mistakes,',
 'compensate',
 'give',
 'insight',
 'use',
 'improvementsactively',
 'communicate',
 'status',
 'expected',
 'resolution',
 'time',
 'availablealign',
 'required',
 'available',
 'crisis',
 'comms',
 'channel',
 'heat',
 'moment.apply',
 'change',
 'management',
 '(roadmap,',
 'cm',
 'process,',
 'program',
 'management)ask',
 'smart',
 'phone',
 'vendor',
 'different',
 'retry',
 'schemeassign',
 'train',
 'deputiesattach',
 'value',
 'improvement',
 'processesbe',
 'front',
 'runner',
 'wellbe',
 'part',
 'research',
 'reportsblock',
 'number',
 'range',
 'release',
 'stepwisebronze',
 'v',
 'gold',
 'quality',
 'service',
 'level.build',
 'crisis',
 'organisationbusiness',
 'continuity',
 'planbuy',
 'toolscentral',
 'contact',
 'listcentral',
 'contact',
 'listcentral',
 'contact',
 'li

In [19]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(data["Recommendation"])

In [20]:
test = bow_transformer.transform([data["Recommendation"]])
test.shape

(1, 485)

In [91]:
tfIdfVectorizer=TfidfTransformer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(np.array(text_process(data["Recommendation"])).reshape(-1, 1))
df = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
df

C:\Users\Michi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:566: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  X = check_array(X, **check_params)


ValueError: Unable to convert array of bytes/strings into decimal numbers with dtype='numeric'